# `scsprox` Tutorial

`scsprox` creates fast proximal operators from CVXPY `Problem` objects.

For this tutorial, we first create a simple CVXPY problem.

In [ ]:
import numpy as np
import cvxpy as cp
from numpy.random import default_rng

m, n = 200, 100

rng = default_rng()
A = rng.standard_normal((m,n))
b = rng.standard_normal(m)
x = cp.Variable(n)

prob = cp.Problem(cp.Minimize(cp.norm(A @ x - b)))
prob.solve()

# The "true" solution, as computed by CVXPY.
x_true = np.array(x.value).flatten()
print("The norm of the residual is {:.3e}.".format(cp.norm(A @ x - b, p=2).value))

# Form the `Prox` object

`scsprox` provides a single object, `Prox`.

We create a `Prox` object by passing in a CVXPY problem, `prob`, along with a dict, `prox_vars`, of the proximal variables.
That is, if `'x'` is a key in `prox_vars`, then we add the proximal regularization $\frac{\rho}{2}\|x-x_0 \|_2^2$
to the objective in `prob` to create the proximal problem.

During initialization, the `Prox` object forms a CySCS `Workspace`, which computes and stores the SCS factorization (which only needs to be computed once).

The `Prox` object accepts arbitrary CVXPY problems and any dict of related CVXPY variables to form the prox.

We set `verbose=True` below to confirm that SCS performs its matrix factorization.

In [ ]:
# For use during development.
#import sys
#import os
#sys.path.append(os.path.abspath('scsprox'))

In [ ]:
%load_ext autoreload
%autoreload 2

from scsprox import Prox

prox_vars = {'x': x}
prox = Prox(prob, prox_vars, verbose=True)

# Evaluate the prox

Below, we'll evaluate the prox using the `Prox._do()` method on the input dict `x0` with `rho=1.0`.
`x0` is a dictionary of variable names and values (matching the names and variable sizes in `prox_vars`).

Note that SCS **doesn't** initialize, because the factorization has been cached,
and that this first call to `Prox._do()` takes 40 iterations.

Again, we make sure `verbose=True` to see the SCS status output.

In [ ]:
x0 = {'x': np.zeros(n)}
rho = 1.0
x1 = prox(x0, rho, verbose=True)

# Prox status

We can see a `dict` of `Prox` status information with `Prox.info`:

In [ ]:
prox.info

# Automatic warm-starting

If we call `prox._do()` again, we can take advantage of warm-starting.
With the same `x0` and `rho` values, we see that the prox completes in 0 SCS iterations!

This is because the SCS solution from the previous `Prox._do()` call is used to warm-start this call.
Since `x0` and `rho` are the same, the SCS problem is identical, and no further iterations are needed.

In [ ]:
x0 = {'x': np.zeros(n)}
rho = 1.0
x1 = prox._do(x0, rho, verbose=True)

# More realistic warm-starting

Of course, we usually won't try to compute the prox on exactly the same value, but instead, a slight perturbation of that value. Warm-starting still helps in this case, and still works automatically.

To see this, we call `Prox.do()` on `x1`, the output of the first prox computation.
SCS is warm-started from the previous solution, which will tend to reduce the number of iterations needed.

In [ ]:
x2 = prox._do(x1, 1.0)

Note that the `Prox.info` dict has been updated.

In [ ]:
prox.info

# Proximal iteration

As an example application, we can solve the original CVXPY problem through proximal iteration.
This involves repeated application of the prox operator.

In [ ]:
for i in range(20):
    x0 = prox._do(x0, 1.0, verbose=False)

Note that, after several iterations, proximal iteration converges, and the SCS solver finishes in **0** iterations.

In [ ]:
prox.info

# Resetting warm-starting

We can also reset the internal warm-start vector to zero, by calling `Prox.reset_warm_start()`.

In [ ]:
prox._do(x0)
prox.info

Note that calling `Prox.reset_warm_start()` increases the number of SCS iterations required to find the solution.

In [ ]:
prox.reset_warm_start()
prox._do(x0, verbose=False)
prox.info

# Prox zero element

The `Prox` object is aware of its input variable names and sizes.
If we call `Prox.do()` without specifying `x0`, or setting it to `{}` or `None`,
the `Prox` object will automatically replace `x0` with the zero element of the
appropriate size, which the user can also access through `Prox.zero_elem`.

In [ ]:
x0 = prox._do(verbose=True)

In [ ]:
prox.zero_elem

# SCS settings

CySCS solver settings can be passed to the `Prox` object either during initialization or through `Prox.do()` as keyword arguments. We've already seen this with the `verbose=True` setting.

Some other common settings are to set the solver tolerance and the maximum number of iterations.

Settings set by the `Prox` constructor or `Prox.do()` persist until explicitly modified again.

In [ ]:
x0 = prox._do(eps=1e-9, verbose=True)

In [ ]:
prox.info

# Problems with several variables

Many problems have more than one array variable for which we'd like to add proximal regularization.
These variables simply need to be named, and included in the `prox_vars` dict input to `Prox(prob, prox_vars)` constructor.

In [ ]:
m, n = 10, 5

A = np.random.randn(m,n)
b = np.random.randn(m)

x = cp.Variable(n)
y = cp.Variable(m)
z = cp.Variable()

obj = cp.sum_squares(A @ x - b) + cp.norm(A.T @ y - x) + 0.1 * cp.norm(y) + cp.norm(z-y)

prob = cp.Problem(cp.Minimize(obj))

In [ ]:
prox_vars = dict(x=x,y=y,z=z)
prox = Prox(prob, prox_vars)

Note that the output to `Prox.do()` is a dict with keys `'x'`, `'y'`, and `'z'`.

`'x'` and `'y'` correspond to `numpy.array` objects, while `'z'` is simply a Python `float`.

In [ ]:
prox._do()

# Many proxes in parallel

Since the underlying CySCS solver releases the Python GIL, we can evaluate many prox operators in parallel using multiple threads. Threads consume less memory than separate Python processes, and do not require data serialization to communicate between processes.

To see this in action, we'll first create several CVXPY problems and their prox operators.

In [ ]:
m, n = 600, 400
k = 10
np.random.seed(0)

x = cp.Variable(n)
xvars = {'x': x}

proxes = []
for _ in range(k):
    A = np.random.randn(m,n)
    b = np.random.randn(m)
    prob = cp.Problem(cp.Minimize(cp.norm(A @ x - b)))
    
    proxes += [Prox(prob,xvars, eps=1e-9)]

We'll use the Python `concurrent.futures` module for our parallel computation.
The module provides a `ThreadPoolExecutor` which has a `map()` method which distributes the computation across multiple threads.

`concurrent.futures` is part of the Python 3 standard library, and is backported to Python 2 via th

To use this interface, we define a simple function `do()` to map over our list of proxes, which does nothing more
than call the `Prox.do()` method. We would need only a slightly more complicated function to pass in specific `x0` values to each prox.

In [ ]:
def do(prox):
    return prox._do()

We create a `ThreadPoolExecutor` with 2 threads, and ensure that the warm-start is reset.

In [ ]:
from concurrent.futures import ThreadPoolExecutor

ex = ThreadPoolExecutor(2)

for prox in proxes:
    prox.reset_warm_start()

We can do the classic (non-parallel) computation by using the Python builtin `map()` function.

Note that this code is equivalent to the list comprehension

```python
output = [prox.do() for prox in proxes]
```

or the for loop

```python
output = []
for prox in proxes:
    output += [prox.do()]
```

We write it as a `map` here because its form is identical to the upcoming parallel computation.

We can time the cell to note the total wall clock time.

In [ ]:
%%time
output = list(map(do, proxes))

Making sure to reset the warm-start variables, we can do the computation in parallel by simply replacing
`map()` with `ex.map()`.

Note that the wall clock time is reduced, since we distribute the computation across two threads (on a dual-core machine).

In [ ]:
for prox in proxes:
    prox.reset_warm_start()

In [ ]:
%%time
output = list(ex.map(do, proxes))

We can see that the sum of the SCS computation times is about twice the parallel wall clock time, which is exactly what we'd expect!

In [ ]:
sum(prox.info['solve_time'] for prox in proxes)